In [1]:
import os
import findspark

os.environ['PATH'] = os.environ['PATH'] + ":/usr/java/jdk1.8.0_162/bin"
os.environ['PYSPARK_PYTHON'] = '/home/tozeng/anaconda3/bin/python'

import findspark
os.environ['PYSPARK_SUBMIT_ARGS'] = \
    "--packages com.databricks:spark-xml_2.11:0.5.0 pyspark-shell"
findspark.init('/opt/cloudera/parcels/SPARK2/lib/spark2/')

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as fn
from pyspark.sql.window import Window

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

spark = SparkSession.builder.\
    appName('sloan-query').\
    config('spark.driver.memory', '20g').\
    config('spark.network.timeout', '600s').\
    config('spark.driver.maxResultSize', '10g').\
    config('spark.executor.memory', '15g').\
    config('spark.kryoserializer.buffer.max', '1g').\
    config('spark.cores.max', '50').\
    getOrCreate()
sc = spark.sparkContext

/home/lliang06/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
dimension_energy_pi = spark.read.parquet('./sloan/dimension_energy_pi.parquet')
authorship = spark.read.parquet('/user/lliang06/daniel-dimensions/authorship.parquet')
potential_researchers = spark.read.parquet('./sloan/potential_researchers.parquet')

In [3]:
import re

In [4]:
authorship = spark.read.parquet('/user/lliang06/daniel-dimensions/authorship.parquet')
citations = spark.read.parquet('/user/lliang06/daniel-dimensions/citations.parquet')
datasets = spark.read.parquet('/user/lliang06/daniel-dimensions/datasets')
clinical_trials = spark.read.parquet('/user/lliang06/daniel-dimensions/clinical_trials')
grants = spark.read.parquet('/user/lliang06/daniel-dimensions/grants')
grid = spark.read.parquet('/user/lliang06/daniel-dimensions/grid')
patents = spark.read.parquet('/user/lliang06/daniel-dimensions/patents')
policy_documents = spark.read.parquet('/user/lliang06/daniel-dimensions/policy_documents')
publications = spark.read.parquet('/user/lliang06/daniel-dimensions/publications')
reports = spark.read.parquet('/user/lliang06/daniel-dimensions/reports')
researchers = spark.read.parquet('/user/lliang06/daniel-dimensions/researchers')
source_titles = spark.read.parquet('/user/lliang06/daniel-dimensions/source_titles')

In [5]:
pubid = pd.read_csv('./data/lda_pubid.csv')

In [12]:
pubid_coauthor_count = spark.createDataFrame(pubid).\
    join(authorship, on = 'publication_id', how = 'inner').\
    groupby('publication_id').count().withColumnRenamed('count', 'coauthor_count').\
    toPandas()

In [13]:
# pubid_coauthor_count.to_csv('./data/pubid_coauthor_count.csv', index = False)

In [14]:
pubid_coauthor_count = pd.read_csv('./data/pubid_coauthor_count.csv')

,publication_id,coauthor_count
0,pub.1000005399,1
1,pub.1000006638,1
2,pub.1000013239,3
3,pub.1000014339,5
4,pub.1000015633,6
...,...,...
6187427,pub.1147433638,4
6187428,pub.1147767156,3
6187429,pub.1147932716,6
6187430,pub.1148007290,6


In [230]:
with open('./data/dup_ids.txt') as f:
    lines = f.readlines()

dimension_dict = {}
dimension_dict['Award_number'] = []
dimension_dict['PI'] = []
dimension_dict['researcher_id'] = []
p = re.compile(r'ur.\d+.\d+')

for l in lines:
    if 'Award_number' in l:
        dimension_dict['Award_number'].append(l.split(' ')[-1][:-1])
    if (len(l.split(' ')) <= 4) & ('Award_number' not in l) & (l is not '\n') & ('combine' not in l) & ('Combine' not in l):
        dimension_dict['PI'].append(l.split('\t')[-1].split('\n')[0])
    if (('combine' in l) | ('Combine' in l)) & ('&' in l):
        if 'combine' in l:
            dimension_dict['researcher_id'].append(l.split('combine')[-1][:-1].split('&'))
        elif 'Combine' in l:
            dimension_dict['researcher_id'].append(l.split('Combine')[-1][:-1].split('&'))
            
            
            

In [267]:
id_mapping = {}
for ids in dimension_dict['researcher_id']:
    for id in ids:
        if id.strip() != '':
            id_mapping[id.strip()] = ids[0].strip()

In [6]:
df = pd.read_csv('./data/energy_grants_Dimensions_wAbstract_cleanest.csv')[['PI', 'researcher_id', 'StartYear', 'EndYear']]

In [7]:
df['YearBefore'] = df['StartYear'] - 1 

In [8]:
df = spark.createDataFrame(df)

In [40]:
df.\
    join(authorship, 'researcher_id', 'inner').\
    select('researcher_id').\
    drop_duplicates().\
    count()

1956

In [41]:
df.count()

2683

In [2]:
from pyspark.sql.functions import row_number
windowSpec  = Window.partitionBy("researcher_id").orderBy("year")

researcher_academic_startyear = authorship.\
    join(df.select('researcher_id'), on = 'researcher_id', how = 'inner').\
    join(publications.select(fn.col('id').alias('publication_id'), 'year'), how = 'inner', on = 'publication_id').\
    dropna().\
    withColumn("row_number", fn.row_number().over(windowSpec)).\
    where(fn.col('row_number') == 1).\
    select('researcher_id', 'year').\
    drop_duplicates().toPandas()

In [37]:
# researcher_academic_startyear.to_csv('./data/researcher_academic_startyear.csv', index = False)
researcher_academic_startyear = pd.read_csv('./data/researcher_academic_startyear.csv')

In [43]:
researcher_academic_startyear_df = spark.createDataFrame(researcher_academic_startyear).\
                                        join(df.select('researcher_id', 'YearBefore'), 'researcher_id', 'inner').\
                                        drop_duplicates()

In [44]:
researcher_academic_startyear_df.count()

2506

### Publication Count

In [45]:
pub_idx_nopp = publications.where(fn.col('type') != 'preprint').select(fn.col('id').alias('publication_id'), 'year')

In [46]:
publication_count = authorship.\
    join(df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    where(fn.col('year') <= fn.col('YearBefore')).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'publication_count')

In [47]:
publication_count_3Y = authorship.\
    join(df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(3)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'publication_count_3Y')

In [48]:
publication_count_5Y = authorship.\
    join(df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(5)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'publication_count_5Y')

In [49]:
publication_count_10Y = authorship.\
    join(df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(10)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'publication_count_10Y')

#### annually

In [50]:
windowSpec  = Window.partitionBy("researcher_id")

researcher_annual = authorship.\
    join(df.select('researcher_id'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    select('researcher_id', 'year').drop_duplicates().\
    withColumn("min", fn.min('year').over(windowSpec)).\
    withColumn("max", fn.max('year').over(windowSpec)).\
    select('researcher_id', 'min', 'max').drop_duplicates().toPandas()

In [51]:
researcher_annual['min'] = researcher_annual['min'].astype(int)
researcher_annual['max'] = researcher_annual['max'].astype(int)

In [52]:
researcher_annual['YearBefore'] = researcher_annual.apply(lambda x: range(x['min'], x['max'] + 1), axis=1)

In [53]:
researcher_annual = researcher_annual.drop(['min', 'max'], axis = 1).explode('YearBefore')

In [54]:
researcher_annual = researcher_annual.dropna().reset_index(drop = True)

In [55]:
researcher_annual['YearBefore'] = researcher_annual['YearBefore'].astype(int)

In [56]:
researcher_annual_df = spark.createDataFrame(researcher_annual)

In [57]:
publication_count_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    where(fn.col('year') <= fn.col('YearBefore')).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'publication_count')

In [58]:
publication_count_3Y_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(3)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'publication_count_3Y')

In [59]:
publication_count_5Y_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(5)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'publication_count_5Y')

In [60]:
publication_count_10Y_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(10)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'publication_count_10Y')

### Citation Received Count by Citing Paper Publication Year

In [61]:
citation_count = authorship.\
    join(df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where(fn.col('citation_year') <= fn.col('YearBefore')).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count')

In [62]:
citation_count_3Y = authorship.\
    join(df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where((fn.col('citation_year') <= fn.col('YearBefore')) & (fn.col('citation_year') > (fn.col('YearBefore') - fn.lit(3)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_3Y')

In [63]:
citation_count_5Y = authorship.\
    join(df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where((fn.col('citation_year') <= fn.col('YearBefore')) & (fn.col('citation_year') > (fn.col('YearBefore') - fn.lit(5)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_5Y')

In [64]:
citation_count_10Y = authorship.\
    join(df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where((fn.col('citation_year') <= fn.col('YearBefore')) & (fn.col('citation_year') > (fn.col('YearBefore') - fn.lit(10)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_10Y')

#### annually

In [65]:
citation_count_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where(fn.col('citation_year') <= fn.col('YearBefore')).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count')

In [66]:
citation_count_3Y_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where((fn.col('citation_year') <= fn.col('YearBefore')) & (fn.col('citation_year') > (fn.col('YearBefore') - fn.lit(3)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_3Y')

In [67]:
citation_count_5Y_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where((fn.col('citation_year') <= fn.col('YearBefore')) & (fn.col('citation_year') > (fn.col('YearBefore') - fn.lit(5)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_5Y')

In [68]:
citation_count_10Y_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where((fn.col('citation_year') <= fn.col('YearBefore')) & (fn.col('citation_year') > (fn.col('YearBefore') - fn.lit(10)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_10Y')

### Citation Received Count by Cited Paper Publication Year

In [69]:
citation_count_3YPub = authorship.\
    join(df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(3)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_3YPub')

In [70]:
citation_count_5YPub = authorship.\
    join(df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(5)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_5YPub')

In [71]:
citation_count_10YPub = authorship.\
    join(df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(10)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_10YPub')

#### annually

In [72]:
citation_count_3YPub_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(3)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_3YPub')

In [73]:
citation_count_5YPub_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(5)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_5YPub')

In [74]:
citation_count_10YPub_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(10)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_10YPub')

### Citation Received Count by Cited Paper Publication Year and by Citing Paper Citing Year

In [75]:
citation_count_3YPubCite = authorship.\
    join(df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id'), fn.col('year').alias('citing_year')), 'citing_publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') >= (fn.col('YearBefore') - fn.lit(3)))).\
    where((fn.col('citing_year') <= fn.col('YearBefore')) & (fn.col('citing_year') > (fn.col('YearBefore') - fn.lit(3)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_3YPubCite')

In [76]:
citation_count_5YPubCite = authorship.\
    join(df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id'), fn.col('year').alias('citing_year')), 'citing_publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') >= (fn.col('YearBefore') - fn.lit(5)))).\
    where((fn.col('citing_year') <= fn.col('YearBefore')) & (fn.col('citing_year') > (fn.col('YearBefore') - fn.lit(5)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_5YPubCite')

In [77]:
citation_count_10YPubCite = authorship.\
    join(df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id'), fn.col('year').alias('citing_year')), 'citing_publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') >= (fn.col('YearBefore') - fn.lit(10)))).\
    where((fn.col('citing_year') <= fn.col('YearBefore')) & (fn.col('citing_year') > (fn.col('YearBefore') - fn.lit(10)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_10YPubCite')

#### annually

In [78]:
citation_count_3YPubCite_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id'), fn.col('year').alias('citing_year')), 'citing_publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') >= (fn.col('YearBefore') - fn.lit(3)))).\
    where((fn.col('citing_year') <= fn.col('YearBefore')) & (fn.col('citing_year') > (fn.col('YearBefore') - fn.lit(3)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_3YPubCite')

In [79]:
citation_count_5YPubCite_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id'), fn.col('year').alias('citing_year')), 'citing_publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') >= (fn.col('YearBefore') - fn.lit(5)))).\
    where((fn.col('citing_year') <= fn.col('YearBefore')) & (fn.col('citing_year') > (fn.col('YearBefore') - fn.lit(5)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_5YPubCite')

In [80]:
citation_count_10YPubCite_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id'), fn.col('year').alias('citing_year')), 'citing_publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') >= (fn.col('YearBefore') - fn.lit(10)))).\
    where((fn.col('citing_year') <= fn.col('YearBefore')) & (fn.col('citing_year') > (fn.col('YearBefore') - fn.lit(10)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_10YPubCite')

### The Number of Unique Co-Authors

In [81]:
coauthor_count = df.\
    select('researcher_id', 'YearBefore').\
    join(authorship, 'researcher_id', 'inner').\
    join(publications.select(fn.col('id').alias('publication_id'), 'year'), 'publication_id', 'inner').\
    where(fn.col('year') <= fn.col('YearBefore')).\
    join(authorship.withColumnRenamed('researcher_id', 'coauthor_id'), 'publication_id', 'inner').\
    where(fn.col('researcher_id') != fn.col('coauthor_id')).\
    select('researcher_id', 'coauthor_id', 'YearBefore').drop_duplicates().\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'coauthor_count')

In [82]:
coauthor_count3Y = df.\
    select('researcher_id', 'YearBefore').\
    join(authorship, 'researcher_id', 'inner').\
    join(publications.select(fn.col('id').alias('publication_id'), 'year'), 'publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(3)))).\
    join(authorship.withColumnRenamed('researcher_id', 'coauthor_id'), 'publication_id', 'inner').\
    where(fn.col('researcher_id') != fn.col('coauthor_id')).\
    select('researcher_id', 'coauthor_id', 'YearBefore').drop_duplicates().\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'coauthor_count3Y')

In [83]:
coauthor_count5Y = df.\
    select('researcher_id', 'YearBefore').\
    join(authorship, 'researcher_id', 'inner').\
    join(publications.select(fn.col('id').alias('publication_id'), 'year'), 'publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(5)))).\
    join(authorship.withColumnRenamed('researcher_id', 'coauthor_id'), 'publication_id', 'inner').\
    where(fn.col('researcher_id') != fn.col('coauthor_id')).\
    select('researcher_id', 'coauthor_id', 'YearBefore').drop_duplicates().\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'coauthor_count5Y')

In [84]:
coauthor_count10Y = df.\
    select('researcher_id', 'YearBefore').\
    join(authorship, 'researcher_id', 'inner').\
    join(publications.select(fn.col('id').alias('publication_id'), 'year'), 'publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(10)))).\
    join(authorship.withColumnRenamed('researcher_id', 'coauthor_id'), 'publication_id', 'inner').\
    where(fn.col('researcher_id') != fn.col('coauthor_id')).\
    select('researcher_id', 'coauthor_id', 'YearBefore').drop_duplicates().\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'coauthor_count10Y')

In [85]:
researcher_bibliometrics = researcher_academic_startyear_df.\
    join(publication_count, ['researcher_id', 'YearBefore'], 'left').\
    join(publication_count_3Y, ['researcher_id', 'YearBefore'], 'left').\
    join(publication_count_5Y, ['researcher_id', 'YearBefore'], 'left').\
    join(publication_count_10Y, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_3Y, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_5Y, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_10Y, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_3YPub, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_5YPub, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_10YPub, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_3YPubCite, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_5YPubCite, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_10YPubCite, ['researcher_id', 'YearBefore'], 'left').\
    join(coauthor_count, ['researcher_id', 'YearBefore'], 'left').\
    join(coauthor_count3Y, ['researcher_id', 'YearBefore'], 'left').\
    join(coauthor_count5Y, ['researcher_id', 'YearBefore'], 'left').\
    join(coauthor_count10Y, ['researcher_id', 'YearBefore'], 'left').\
    fillna(0).toPandas()

In [86]:
researcher_bibliometrics.to_csv('./data/researcher_bibliometrics.csv', index = False)

In [6]:
researcher_bibliometrics = pd.read_csv('../sloan/data/researcher_bibliometrics.csv')

#### annually

In [87]:
coauthor_count_annual = researcher_annual_df.\
    select('researcher_id', 'YearBefore').\
    join(authorship, 'researcher_id', 'inner').\
    join(publications.select(fn.col('id').alias('publication_id'), 'year'), 'publication_id', 'inner').\
    where(fn.col('year') <= fn.col('YearBefore')).\
    join(authorship.withColumnRenamed('researcher_id', 'coauthor_id'), 'publication_id', 'inner').\
    where(fn.col('researcher_id') != fn.col('coauthor_id')).\
    select('researcher_id', 'coauthor_id', 'YearBefore').drop_duplicates().\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'coauthor_count')

In [88]:
coauthor_count3Y_annual = researcher_annual_df.\
    select('researcher_id', 'YearBefore').\
    join(authorship, 'researcher_id', 'inner').\
    join(publications.select(fn.col('id').alias('publication_id'), 'year'), 'publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(3)))).\
    join(authorship.withColumnRenamed('researcher_id', 'coauthor_id'), 'publication_id', 'inner').\
    where(fn.col('researcher_id') != fn.col('coauthor_id')).\
    select('researcher_id', 'coauthor_id', 'YearBefore').drop_duplicates().\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'coauthor_count3Y')

In [89]:
coauthor_count5Y_annual = researcher_annual_df.\
    select('researcher_id', 'YearBefore').\
    join(authorship, 'researcher_id', 'inner').\
    join(publications.select(fn.col('id').alias('publication_id'), 'year'), 'publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(5)))).\
    join(authorship.withColumnRenamed('researcher_id', 'coauthor_id'), 'publication_id', 'inner').\
    where(fn.col('researcher_id') != fn.col('coauthor_id')).\
    select('researcher_id', 'coauthor_id', 'YearBefore').drop_duplicates().\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'coauthor_count5Y')

In [90]:
coauthor_count10Y_annual = researcher_annual_df.\
    select('researcher_id', 'YearBefore').\
    join(authorship, 'researcher_id', 'inner').\
    join(publications.select(fn.col('id').alias('publication_id'), 'year'), 'publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(10)))).\
    join(authorship.withColumnRenamed('researcher_id', 'coauthor_id'), 'publication_id', 'inner').\
    where(fn.col('researcher_id') != fn.col('coauthor_id')).\
    select('researcher_id', 'coauthor_id', 'YearBefore').drop_duplicates().\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'coauthor_count10Y')

In [91]:
researcher_bibliometrics_annual = researcher_annual_df.\
    join(publication_count_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(publication_count_3Y_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(publication_count_5Y_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(publication_count_10Y_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_3Y_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_5Y_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_10Y_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_3YPub_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_5YPub_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_10YPub_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_3YPubCite_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_5YPubCite_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_10YPubCite_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(coauthor_count_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(coauthor_count3Y_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(coauthor_count5Y_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(coauthor_count10Y_annual, ['researcher_id', 'YearBefore'], 'left').\
    fillna(0).toPandas()

In [92]:
researcher_bibliometrics_annual.sort_values(by = ['researcher_id', 'YearBefore']).to_csv('./data/researcher_annual_bibliometrics.csv', index = False)

In [8]:
researcher_bibliometrics_annual = pd.read_csv('../sloan/data/researcher_annual_bibliometrics.csv')

In [9]:
researcher_bibliometrics_annual

,researcher_id,YearBefore,publication_count,publication_count_3Y,publication_count_5Y,publication_count_10Y,citation_count,citation_count_3Y,citation_count_5Y,citation_count_10Y,citation_count_3YPub,citation_count_5YPub,citation_count_10YPub,citation_count_3YPubCite,citation_count_5YPubCite,citation_count_10YPubCite,coauthor_count,coauthor_count3Y,coauthor_count5Y,coauthor_count10Y
0,ur.01000273107.88,1981,2,2,2,2,0,0,0,0,0,0,0,0,0,0,2,2,2,2
1,ur.01000273107.88,1982,2,2,2,2,0,0,0,0,0,0,0,0,0,0,2,2,2,2
2,ur.01000273107.88,1983,4,4,4,4,0,0,0,0,0,0,0,0,0,0,3,3,3,3
3,ur.01000273107.88,1984,4,2,4,4,0,0,0,0,0,0,0,0,0,0,3,3,3,3
4,ur.01000273107.88,1985,5,3,5,5,0,0,0,0,0,0,0,0,0,0,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54684,ur.0777742233.90,2017,427,46,79,139,11461,2368,3772,7057,654,1341,2597,333,733,1601,321,76,98,141
54685,ur.0777742233.90,2018,433,33,68,132,12204,2362,3825,7111,266,904,2248,333,823,1801,328,58,94,135
54686,ur.0777742233.90,2019,444,27,63,133,12874,2201,3781,7206,137,747,2172,149,637,1729,333,40,91,135
54687,ur.0777742233.90,2020,449,22,49,125,13475,2014,3633,7111,127,353,1962,85,575,1850,341,41,72,133


# Bib for potential researchers

In [11]:
potential_researchers = spark.read.parquet('./sloan/potential_researchers.parquet')

In [12]:
from pyspark.sql.functions import row_number
windowSpec  = Window.partitionBy("researcher_id").orderBy("year")

researcher_academic_startyear = authorship.\
    join(potential_researchers.select('researcher_id'), on = 'researcher_id', how = 'inner').\
    join(publications.select(fn.col('id').alias('publication_id'), 'year'), how = 'inner', on = 'publication_id').\
    dropna().\
    withColumn("row_number", fn.row_number().over(windowSpec)).\
    where(fn.col('row_number') == 1).\
    select('researcher_id', 'year').\
    drop_duplicates().toPandas()

In [13]:
# researcher_academic_startyear.to_csv('./data/potential_researcher_academic_startyear.csv', index = False)
researcher_academic_startyear = pd.read_csv('./data/potential_researcher_academic_startyear.csv')

### Publication Count

In [71]:
pub_idx_nopp = publications.where(fn.col('type') != 'preprint').select(fn.col('id').alias('publication_id'), 'year')

In [100]:
windowSpec  = Window.partitionBy("researcher_id")

researcher_annual = authorship.\
    join(potential_researchers.select('researcher_id'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    select('researcher_id', 'year').drop_duplicates().\
    withColumn("min", fn.min('year').over(windowSpec)).\
    withColumn("max", fn.max('year').over(windowSpec)).\
    select('researcher_id', 'min', 'max').drop_duplicates().toPandas()

In [106]:
researcher_annual['min'] = researcher_annual['min'].astype(int)
researcher_annual['max'] = researcher_annual['max'].astype(int)

In [107]:
researcher_annual['YearBefore'] = researcher_annual.apply(lambda x: range(x['min'], x['max'] + 1), axis=1)

In [108]:
researcher_annual = researcher_annual.drop(['min', 'max'], axis = 1).explode('YearBefore')

In [109]:
researcher_annual = researcher_annual.dropna().reset_index(drop = True)

In [110]:
researcher_annual['YearBefore'] = researcher_annual['YearBefore'].astype(int)

In [111]:
researcher_annual_df = spark.createDataFrame(researcher_annual)

In [113]:
researcher_annual_df.write.parquet('./sloan/potential_researcher_annual.parquet', mode = 'overwrite')

In [114]:
researcher_annual_df = spark.read.parquet('./sloan/potential_researcher_annual.parquet')

In [105]:
researcher_annual_df[researcher_annual_df['researcher_id'] == 'ur.01000000322.68'].show()

+-----------------+----------+
|    researcher_id|YearBefore|
+-----------------+----------+
|ur.01000000322.68|      2014|
|ur.01000000322.68|      2015|
|ur.01000000322.68|      2016|
|ur.01000000322.68|      2017|
|ur.01000000322.68|      2018|
|ur.01000000322.68|      2019|
|ur.01000000322.68|      2020|
+-----------------+----------+



In [115]:
researcher_annual_df[researcher_annual_df['researcher_id'] == 'ur.01000000322.68'].show()

+-----------------+----------+
|    researcher_id|YearBefore|
+-----------------+----------+
|ur.01000000322.68|      2014|
|ur.01000000322.68|      2015|
|ur.01000000322.68|      2016|
|ur.01000000322.68|      2017|
|ur.01000000322.68|      2018|
|ur.01000000322.68|      2019|
|ur.01000000322.68|      2020|
|ur.01000000322.68|      2021|
+-----------------+----------+



In [136]:
publication_count_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    where(fn.col('year') <= fn.col('YearBefore')).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'publication_count')

In [139]:
publication_count_1Y_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(1)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'publication_count_1Y')

In [117]:
publication_count_3Y_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(3)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'publication_count_3Y')

In [118]:
publication_count_5Y_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(5)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'publication_count_5Y')

In [119]:
publication_count_10Y_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(10)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'publication_count_10Y')

### Citation Received Count by Citing Paper Publication Year

In [120]:
citation_count_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where(fn.col('citation_year') <= fn.col('YearBefore')).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count')

In [121]:
citation_count_3Y_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where((fn.col('citation_year') <= fn.col('YearBefore')) & (fn.col('citation_year') > (fn.col('YearBefore') - fn.lit(3)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_3Y')

In [122]:
citation_count_5Y_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where((fn.col('citation_year') <= fn.col('YearBefore')) & (fn.col('citation_year') > (fn.col('YearBefore') - fn.lit(5)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_5Y')

In [123]:
citation_count_10Y_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where((fn.col('citation_year') <= fn.col('YearBefore')) & (fn.col('citation_year') > (fn.col('YearBefore') - fn.lit(10)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_10Y')

### Citation Received Count by Cited Paper Publication Year

In [124]:
citation_count_3YPub_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(3)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_3YPub')

In [125]:
citation_count_5YPub_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(5)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_5YPub')

In [126]:
citation_count_10YPub_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id')), 'citing_publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(10)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_10YPub')

### Citation Received Count by Cited Paper Publication Year and by Citing Paper Citing Year

In [127]:
citation_count_3YPubCite_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id'), fn.col('year').alias('citing_year')), 'citing_publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(3)))).\
    where((fn.col('citing_year') <= fn.col('YearBefore')) & (fn.col('citing_year') > (fn.col('YearBefore') - fn.lit(3)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_3YPubCite')

In [128]:
citation_count_5YPubCite_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id'), fn.col('year').alias('citing_year')), 'citing_publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(5)))).\
    where((fn.col('citing_year') <= fn.col('YearBefore')) & (fn.col('citing_year') > (fn.col('YearBefore') - fn.lit(5)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_5YPubCite')

In [129]:
citation_count_10YPubCite_annual = authorship.\
    join(researcher_annual_df.select('researcher_id', 'YearBefore'), on = 'researcher_id', how = 'inner').\
    join(pub_idx_nopp, how = 'inner', on = 'publication_id').\
    join(citations, 'publication_id', 'inner').\
    join(pub_idx_nopp.select(fn.col('publication_id').alias('citing_publication_id'), fn.col('year').alias('citing_year')), 'citing_publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(10)))).\
    where((fn.col('citing_year') <= fn.col('YearBefore')) & (fn.col('citing_year') > (fn.col('YearBefore') - fn.lit(10)))).\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'citation_count_10YPubCite')

### The Number of Unique Co-Authors

In [130]:
coauthor_count_annual = researcher_annual_df.\
    select('researcher_id', 'YearBefore').\
    join(authorship, 'researcher_id', 'inner').\
    join(publications.select(fn.col('id').alias('publication_id'), 'year'), 'publication_id', 'inner').\
    where(fn.col('year') <= fn.col('YearBefore')).\
    join(authorship.withColumnRenamed('researcher_id', 'coauthor_id'), 'publication_id', 'inner').\
    where(fn.col('researcher_id') != fn.col('coauthor_id')).\
    select('researcher_id', 'coauthor_id', 'YearBefore').drop_duplicates().\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'coauthor_count')

In [131]:
coauthor_count3Y_annual = researcher_annual_df.\
    select('researcher_id', 'YearBefore').\
    join(authorship, 'researcher_id', 'inner').\
    join(publications.select(fn.col('id').alias('publication_id'), 'year'), 'publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(3)))).\
    join(authorship.withColumnRenamed('researcher_id', 'coauthor_id'), 'publication_id', 'inner').\
    where(fn.col('researcher_id') != fn.col('coauthor_id')).\
    select('researcher_id', 'coauthor_id', 'YearBefore').drop_duplicates().\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'coauthor_count3Y')

In [132]:
coauthor_count5Y_annual = researcher_annual_df.\
    select('researcher_id', 'YearBefore').\
    join(authorship, 'researcher_id', 'inner').\
    join(publications.select(fn.col('id').alias('publication_id'), 'year'), 'publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(5)))).\
    join(authorship.withColumnRenamed('researcher_id', 'coauthor_id'), 'publication_id', 'inner').\
    where(fn.col('researcher_id') != fn.col('coauthor_id')).\
    select('researcher_id', 'coauthor_id', 'YearBefore').drop_duplicates().\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'coauthor_count5Y')

In [133]:
coauthor_count10Y_annual = researcher_annual_df.\
    select('researcher_id', 'YearBefore').\
    join(authorship, 'researcher_id', 'inner').\
    join(publications.select(fn.col('id').alias('publication_id'), 'year'), 'publication_id', 'inner').\
    where((fn.col('year') <= fn.col('YearBefore')) & (fn.col('year') > (fn.col('YearBefore') - fn.lit(10)))).\
    join(authorship.withColumnRenamed('researcher_id', 'coauthor_id'), 'publication_id', 'inner').\
    where(fn.col('researcher_id') != fn.col('coauthor_id')).\
    select('researcher_id', 'coauthor_id', 'YearBefore').drop_duplicates().\
    groupby('researcher_id', 'YearBefore').count().withColumnRenamed('count', 'coauthor_count10Y')

In [140]:
researcher_bibliometrics_annual = researcher_annual_df.\
    join(publication_count_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(publication_count_1Y_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(publication_count_3Y_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(publication_count_5Y_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(publication_count_10Y_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_3Y_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_5Y_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_10Y_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_3YPub_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_5YPub_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_10YPub_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_3YPubCite_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_5YPubCite_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(citation_count_10YPubCite_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(coauthor_count_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(coauthor_count3Y_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(coauthor_count5Y_annual, ['researcher_id', 'YearBefore'], 'left').\
    join(coauthor_count10Y_annual, ['researcher_id', 'YearBefore'], 'left').\
    withColumnRenamed('YearBefore', 'year').\
    fillna(0)

In [141]:
researcher_bibliometrics_annual.write.parquet('./sloan/potential_researcher_bibliometrics_annual.parquet', mode = 'overwrite')

In [142]:
researcher_bibliometrics_annual = spark.read.parquet('./sloan/potential_researcher_bibliometrics_annual.parquet')

In [144]:
researcher_bibliometrics_annual_df = researcher_bibliometrics_annual.\
    orderBy(['researcher_id', 'year']).\
    toPandas()

In [145]:
researcher_bibliometrics_annual_df.to_csv('./data/potential_researcher_annual_bibliometrics.csv', index = False)

In [146]:
researcher_bibliometrics_annual_df = pd.read_csv('./data/potential_researcher_annual_bibliometrics.csv')

In [147]:
researcher_bibliometrics_annual_df

,researcher_id,year,publication_count,publication_count_1Y,publication_count_3Y,publication_count_5Y,publication_count_10Y,citation_count,citation_count_3Y,citation_count_5Y,...,citation_count_3YPub,citation_count_5YPub,citation_count_10YPub,citation_count_3YPubCite,citation_count_5YPubCite,citation_count_10YPubCite,coauthor_count,coauthor_count3Y,coauthor_count5Y,coauthor_count10Y
0,ur.01000000322.68,2014,1,1,1,1,1,5,5,5,...,55,55,55,5,5,5,16,16,16,16
1,ur.01000000322.68,2015,1,0,1,1,1,10,10,10,...,55,55,55,10,10,10,16,16,16,16
2,ur.01000000322.68,2016,3,2,3,3,3,20,20,20,...,70,70,70,20,20,20,28,28,28,28
3,ur.01000000322.68,2017,4,1,3,4,4,32,27,32,...,378,433,433,4,32,32,36,20,36,36
4,ur.01000000322.68,2018,5,1,4,5,5,102,92,102,...,705,760,760,68,102,102,43,27,43,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5147230,ur.0777777774.33,2015,15,4,6,8,15,202,134,178,...,995,1048,1202,63,89,201,48,33,37,48
5147231,ur.0777777774.33,2016,21,6,12,14,21,319,219,270,...,1186,1239,1393,166,198,318,64,50,54,64
5147232,ur.0777777774.33,2017,21,0,10,12,18,488,359,420,...,283,1186,1370,83,321,468,64,41,50,59
5147233,ur.0777777774.33,2018,22,1,7,13,19,645,443,545,...,202,1197,1381,72,466,624,68,38,54,63


In [57]:
potential_author_bib = pd.read_csv('./data/potential_author_bib.csv')

In [96]:
len(potential_author_bib['related_researcher_id'].unique())

5155861

In [60]:
potential_author_bib.\
    groupby('researcher_id').count()

,related_researcher_id
researcher_id,
ur.01000273107.88,30173
ur.01000501355.27,16368
ur.010006461352.02,32290
ur.010006462741.55,6256
ur.01001040264.31,36068
...,...
ur.0777277254.78,16913
ur.07773074402.04,16983
ur.0777524705.86,97617


In [61]:
df = _

In [62]:
df['related_researcher_id'].max()

225603